# Importing Libraries

In [49]:
import os
import re
import glob
import pandas
import warnings
import pytesseract
import pandas as pd
from PIL import Image
from docx import Document
from dateutil import parser
from datetime import datetime
from pptx import Presentation
from hurry.filesize import size
warnings.filterwarnings('ignore')
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
from pdf2image import convert_from_path
poppler_path = r"poppler-0.68.0\bin"
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\Dinesh.murugan\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

# Meta-Data Extraction

In [50]:
path_dict = {
    "Path" : [],
    "FileName" : [],
    "FileType" : [],
    "ModifiedTime" : [],
    "CreationTime" : [],
    "FileSize" : []
}

root_exp_path = "Dataset/*.*"
for exp_path in [root_exp_path]:
        for path in glob.glob(exp_path):
            path_dict["Path"].append(path)
            path_dict["FileName"].append(path.split("\\")[-1:][0])
            path_dict["FileType"].append(path.split("\\")[-1:][0].split(".")[-1])
            timestamp = os.path.getmtime(path)
            path_dict["ModifiedTime"].append(datetime.fromtimestamp(timestamp))
            timestamp = os.path.getctime(path)
            path_dict["CreationTime"].append(datetime.fromtimestamp(timestamp))
            file_stat = os.stat(path)
            path_dict['FileSize'].append((file_stat.st_size))
path_df = pd.DataFrame.from_dict(path_dict)

In [51]:
path_df["Textfile_Name"] = path_df.FileName + ".txt"

In [52]:
path_df['ModifiedTime'] = pd.DatetimeIndex(path_df.ModifiedTime).date
path_df['CreationTime'] = pd.DatetimeIndex(path_df.CreationTime).date

In [53]:
path_df.head(5)

,Path,FileName,FileType,ModifiedTime,CreationTime,FileSize,Textfile_Name
0,Dataset\2021 Crytpocurrencies_Presentation.pdf,2021 Crytpocurrencies_Presentation.pdf,pdf,2022-10-11,2022-10-11,838394,2021 Crytpocurrencies_Presentation.pdf.txt
1,Dataset\@One-pager How do higher rates slow th...,@One-pager How do higher rates slow the econom...,docx,2022-10-11,2022-10-11,98091,@One-pager How do higher rates slow the econom...
2,Dataset\@One-pager What is a Bull Market.docx,@One-pager What is a Bull Market.docx,docx,2022-10-11,2022-10-11,122763,@One-pager What is a Bull Market.docx.txt
3,Dataset\@One-pager Why Invest Globally_NB.docx,@One-pager Why Invest Globally_NB.docx,docx,2022-10-11,2022-10-11,107097,@One-pager Why Invest Globally_NB.docx.txt
4,Dataset\@One-pager Why Invest in Hedge Funds.docx,@One-pager Why Invest in Hedge Funds.docx,docx,2022-10-11,2022-10-11,100012,@One-pager Why Invest in Hedge Funds.docx.txt


In [54]:
path_df.FileType.value_counts()

pdf     16
docx     4
pptx     3
Name: FileType, dtype: int64

In [55]:
path_df.FileType.value_counts().index.tolist()

['pdf', 'docx', 'pptx']

In [56]:
path_df.to_csv(r"C:\Datascience\Meketa\path.csv", index=False)

# Preparing min-max values for searching

In [57]:
filetypes = path_df.FileType.value_counts().index.tolist()
modification_time_range = [min(path_df.ModifiedTime), max(path_df.ModifiedTime)]
creation_time_range = [min(path_df.CreationTime), max(path_df.CreationTime)]
file_size_range = [min(path_df.FileSize), max(path_df.FileSize)]

In [58]:
str(min(path_df.ModifiedTime)).split(" ")[0]

'2022-10-11'

In [59]:
type(min(pd.DatetimeIndex(path_df.ModifiedTime).date))

datetime.date

In [60]:
def convert_bytes(bytes_number):
    tags = [ "Byte", "KB", "MB", "GB", "TB" ]
    i = 0
    double_bytes = bytes_number
    while (i < len(tags) and  bytes_number >= 1024):
            double_bytes = bytes_number / 1024.0
            i = i + 1
            bytes_number = bytes_number / 1024
    return str(round(double_bytes, 2)) + " " + tags[i]
print(convert_bytes(838394))

818.74 KB


In [61]:
path_df["File_Size_ORG"] = [convert_bytes(x) for x in path_df.FileSize]

In [62]:
path_df

,Path,FileName,FileType,ModifiedTime,CreationTime,FileSize,Textfile_Name,File_Size_ORG
0,Dataset\2021 Crytpocurrencies_Presentation.pdf,2021 Crytpocurrencies_Presentation.pdf,pdf,2022-10-11,2022-10-11,838394,2021 Crytpocurrencies_Presentation.pdf.txt,818.74 KB
1,Dataset\@One-pager How do higher rates slow th...,@One-pager How do higher rates slow the econom...,docx,2022-10-11,2022-10-11,98091,@One-pager How do higher rates slow the econom...,95.79 KB
2,Dataset\@One-pager What is a Bull Market.docx,@One-pager What is a Bull Market.docx,docx,2022-10-11,2022-10-11,122763,@One-pager What is a Bull Market.docx.txt,119.89 KB
3,Dataset\@One-pager Why Invest Globally_NB.docx,@One-pager Why Invest Globally_NB.docx,docx,2022-10-11,2022-10-11,107097,@One-pager Why Invest Globally_NB.docx.txt,104.59 KB
4,Dataset\@One-pager Why Invest in Hedge Funds.docx,@One-pager Why Invest in Hedge Funds.docx,docx,2022-10-11,2022-10-11,100012,@One-pager Why Invest in Hedge Funds.docx.txt,97.67 KB
5,Dataset\Core Real Estate WP.pdf,Core Real Estate WP.pdf,pdf,2022-10-11,2022-10-11,357552,Core Real Estate WP.pdf.txt,349.17 KB
6,Dataset\Cycles in Active Management WP.pdf,Cycles in Active Management WP.pdf,pdf,2022-10-11,2022-10-11,760476,Cycles in Active Management WP.pdf.txt,742.65 KB
7,Dataset\Dual Portfolio Framework FINAL.pdf,Dual Portfolio Framework FINAL.pdf,pdf,2022-10-11,2022-10-11,509218,Dual Portfolio Framework FINAL.pdf.txt,497.28 KB
8,Dataset\EM ILBs WP.pdf,EM ILBs WP.pdf,pdf,2022-10-11,2022-10-11,151407,EM ILBs WP.pdf.txt,147.86 KB
9,Dataset\FrontierMarkets.pptx,FrontierMarkets.pptx,pptx,2022-10-11,2022-10-11,1489780,FrontierMarkets.pptx.txt,1.42 MB


# Meta-Data Filtering

In [63]:
# Descending Order of ModifiedTime, CreationTime
path_df.sort_values(by=['ModifiedTime'] , ascending=False)

# Ascending Order of ModifiedTime
path_df.sort_values(by=['ModifiedTime'] , ascending=True)

,Path,FileName,FileType,ModifiedTime,CreationTime,FileSize,Textfile_Name,File_Size_ORG
0,Dataset\2021 Crytpocurrencies_Presentation.pdf,2021 Crytpocurrencies_Presentation.pdf,pdf,2022-10-11,2022-10-11,838394,2021 Crytpocurrencies_Presentation.pdf.txt,818.74 KB
20,Dataset\MEKETA_Short Term TIPS Presentation.pptx,MEKETA_Short Term TIPS Presentation.pptx,pptx,2022-10-11,2022-10-11,15526637,MEKETA_Short Term TIPS Presentation.pptx.txt,14.81 MB
19,Dataset\MEKETA_Portable Alpha.pdf,MEKETA_Portable Alpha.pdf,pdf,2022-10-11,2022-10-11,103113,MEKETA_Portable Alpha.pdf.txt,100.7 KB
18,Dataset\MEKETA_Impact of Convenant-Lite.pdf,MEKETA_Impact of Convenant-Lite.pdf,pdf,2022-10-11,2022-10-11,146506,MEKETA_Impact of Convenant-Lite.pdf.txt,143.07 KB
17,Dataset\MEKETA_Global Equity Mandates.pdf,MEKETA_Global Equity Mandates.pdf,pdf,2022-10-11,2022-10-11,297096,MEKETA_Global Equity Mandates.pdf.txt,290.13 KB
16,Dataset\MEKETA_Co-investment Program Primer.pdf,MEKETA_Co-investment Program Primer.pdf,pdf,2022-10-11,2022-10-11,108062,MEKETA_Co-investment Program Primer.pdf.txt,105.53 KB
15,Dataset\Manager Alpha FINAL.pdf,Manager Alpha FINAL.pdf,pdf,2022-10-11,2022-10-11,915014,Manager Alpha FINAL.pdf.txt,893.57 KB
14,Dataset\Long-Term Treasuries WP.pdf,Long-Term Treasuries WP.pdf,pdf,2022-10-11,2022-10-11,579867,Long-Term Treasuries WP.pdf.txt,566.28 KB
13,Dataset\Introduction to Private Equity present...,Introduction to Private Equity presentation.pptx,pptx,2022-10-11,2022-10-11,198007,Introduction to Private Equity presentation.pp...,193.37 KB
12,Dataset\High Yield Bonds FINAL.pdf,High Yield Bonds FINAL.pdf,pdf,2022-10-11,2022-10-11,802328,High Yield Bonds FINAL.pdf.txt,783.52 KB


In [64]:
# Ascending Order of FileSize
path_df.sort_values(by=['FileSize'] , ascending=True)

# Descending Order of FileSize
path_df.sort_values(by=['FileSize'] , ascending=False)

,Path,FileName,FileType,ModifiedTime,CreationTime,FileSize,Textfile_Name,File_Size_ORG
20,Dataset\MEKETA_Short Term TIPS Presentation.pptx,MEKETA_Short Term TIPS Presentation.pptx,pptx,2022-10-11,2022-10-11,15526637,MEKETA_Short Term TIPS Presentation.pptx.txt,14.81 MB
11,Dataset\High Dimension Optimization 6.11.18.pdf,High Dimension Optimization 6.11.18.pdf,pdf,2022-10-11,2022-10-11,1731650,High Dimension Optimization 6.11.18.pdf.txt,1.65 MB
9,Dataset\FrontierMarkets.pptx,FrontierMarkets.pptx,pptx,2022-10-11,2022-10-11,1489780,FrontierMarkets.pptx.txt,1.42 MB
15,Dataset\Manager Alpha FINAL.pdf,Manager Alpha FINAL.pdf,pdf,2022-10-11,2022-10-11,915014,Manager Alpha FINAL.pdf.txt,893.57 KB
0,Dataset\2021 Crytpocurrencies_Presentation.pdf,2021 Crytpocurrencies_Presentation.pdf,pdf,2022-10-11,2022-10-11,838394,2021 Crytpocurrencies_Presentation.pdf.txt,818.74 KB
12,Dataset\High Yield Bonds FINAL.pdf,High Yield Bonds FINAL.pdf,pdf,2022-10-11,2022-10-11,802328,High Yield Bonds FINAL.pdf.txt,783.52 KB
6,Dataset\Cycles in Active Management WP.pdf,Cycles in Active Management WP.pdf,pdf,2022-10-11,2022-10-11,760476,Cycles in Active Management WP.pdf.txt,742.65 KB
10,Dataset\HealthCare WP.pdf,HealthCare WP.pdf,pdf,2022-10-11,2022-10-11,662222,HealthCare WP.pdf.txt,646.7 KB
21,"Dataset\MEKETA_Understanding China, Part II.pdf","MEKETA_Understanding China, Part II.pdf",pdf,2022-10-11,2022-10-11,636591,"MEKETA_Understanding China, Part II.pdf.txt",621.67 KB
14,Dataset\Long-Term Treasuries WP.pdf,Long-Term Treasuries WP.pdf,pdf,2022-10-11,2022-10-11,579867,Long-Term Treasuries WP.pdf.txt,566.28 KB


In [65]:
# Descending Order of ModifiedTime
path_df.sort_values(by=['FileType'])

,Path,FileName,FileType,ModifiedTime,CreationTime,FileSize,Textfile_Name,File_Size_ORG
1,Dataset\@One-pager How do higher rates slow th...,@One-pager How do higher rates slow the econom...,docx,2022-10-11,2022-10-11,98091,@One-pager How do higher rates slow the econom...,95.79 KB
2,Dataset\@One-pager What is a Bull Market.docx,@One-pager What is a Bull Market.docx,docx,2022-10-11,2022-10-11,122763,@One-pager What is a Bull Market.docx.txt,119.89 KB
3,Dataset\@One-pager Why Invest Globally_NB.docx,@One-pager Why Invest Globally_NB.docx,docx,2022-10-11,2022-10-11,107097,@One-pager Why Invest Globally_NB.docx.txt,104.59 KB
4,Dataset\@One-pager Why Invest in Hedge Funds.docx,@One-pager Why Invest in Hedge Funds.docx,docx,2022-10-11,2022-10-11,100012,@One-pager Why Invest in Hedge Funds.docx.txt,97.67 KB
0,Dataset\2021 Crytpocurrencies_Presentation.pdf,2021 Crytpocurrencies_Presentation.pdf,pdf,2022-10-11,2022-10-11,838394,2021 Crytpocurrencies_Presentation.pdf.txt,818.74 KB
19,Dataset\MEKETA_Portable Alpha.pdf,MEKETA_Portable Alpha.pdf,pdf,2022-10-11,2022-10-11,103113,MEKETA_Portable Alpha.pdf.txt,100.7 KB
18,Dataset\MEKETA_Impact of Convenant-Lite.pdf,MEKETA_Impact of Convenant-Lite.pdf,pdf,2022-10-11,2022-10-11,146506,MEKETA_Impact of Convenant-Lite.pdf.txt,143.07 KB
17,Dataset\MEKETA_Global Equity Mandates.pdf,MEKETA_Global Equity Mandates.pdf,pdf,2022-10-11,2022-10-11,297096,MEKETA_Global Equity Mandates.pdf.txt,290.13 KB
16,Dataset\MEKETA_Co-investment Program Primer.pdf,MEKETA_Co-investment Program Primer.pdf,pdf,2022-10-11,2022-10-11,108062,MEKETA_Co-investment Program Primer.pdf.txt,105.53 KB
15,Dataset\Manager Alpha FINAL.pdf,Manager Alpha FINAL.pdf,pdf,2022-10-11,2022-10-11,915014,Manager Alpha FINAL.pdf.txt,893.57 KB


# Content Extration

In [8]:
for x in path_df.Path:
    file_name = x.split("\\")[-1:][0]
    if x.split("\\")[-1:][0].split(".")[-1] == "pdf":
        images = convert_from_path(x, poppler_path=poppler_path)

        str_ext_global = ""

        for i in range(len(images)):
            str_ext = pytesseract.image_to_string(images[i], lang='eng')
            str_ext_global += str_ext

        with open("Dataset/Textset/" + file_name +".txt", 'w') as f:
            f.write(str_ext_global)
            print(x, ">>> Write Successful")

Dataset\2021 Crytpocurrencies_Presentation.pdf >>> Write Successful
Dataset\Core Real Estate WP.pdf >>> Write Successful
Dataset\Cycles in Active Management WP.pdf >>> Write Successful
Dataset\Dual Portfolio Framework FINAL.pdf >>> Write Successful
Dataset\EM ILBs WP.pdf >>> Write Successful
Dataset\HealthCare WP.pdf >>> Write Successful
Dataset\High Dimension Optimization 6.11.18.pdf >>> Write Successful
Dataset\High Yield Bonds FINAL.pdf >>> Write Successful
Dataset\Long-Term Treasuries WP.pdf >>> Write Successful
Dataset\Manager Alpha FINAL.pdf >>> Write Successful
Dataset\MEKETA_Co-investment Program Primer.pdf >>> Write Successful
Dataset\MEKETA_Global Equity Mandates.pdf >>> Write Successful
Dataset\MEKETA_Impact of Convenant-Lite.pdf >>> Write Successful
Dataset\MEKETA_Portable Alpha.pdf >>> Write Successful
Dataset\MEKETA_Understanding China, Part II.pdf >>> Write Successful
Dataset\Stable Value WP.pdf >>> Write Successful


In [15]:
for x in path_df.Path:
    file_name = x.split("\\")[-1:][0]
    if (x.split("\\")[-1:][0].split(".")[-1]).lower() == "pptx":
        prs = Presentation(x)
        ppt_ext_global = ""
        for slide in prs.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    ppt_ext_global += shape.text
                    
        with open("Dataset/Textset/" + file_name +".txt", 'w') as f:
            f.write(ppt_ext_global)
            print(x, ">>> Write Successful")

Dataset\FrontierMarkets.pptx >>> Write Successful
Dataset\Introduction to Private Equity presentation.pptx >>> Write Successful
Dataset\MEKETA_Short Term TIPS Presentation.pptx >>> Write Successful


In [112]:
for x in path_df.Path:
    file_name = x.split("\\")[-1:][0]
    if (x.split("\\")[-1:][0].split(".")[-1]).lower() == "docx":
        document = Document(x)
        
        docx_ext_global = ""
        try:
            table = document.tables[0]
            data = []
            keys = None
            for i, row in enumerate(table.rows):
                for cell in row.cells:
                    docx_ext_global += "\n" + (cell.text)
        except:
            pass

        for p in document.paragraphs:
            docx_ext_global += "\n" + p.text
        
                    
        with open("Dataset/Textset/" + file_name +".txt", 'w') as f:
            f.write(docx_ext_global)
            print(x, ">>> Write Successful")

Dataset\@One-pager How do higher rates slow the economy.docx >>> Write Successful
Dataset\@One-pager What is a Bull Market.docx >>> Write Successful
Dataset\@One-pager Why Invest Globally_NB.docx >>> Write Successful
Dataset\@One-pager Why Invest in Hedge Funds.docx >>> Write Successful


# Keywork Search in File Names

In [66]:
keyword = "invest"

In [67]:
file_size_range, creation_time_range, modification_time_range, filetypes

([98091, 15526637],
 [datetime.date(2022, 10, 11), datetime.date(2022, 10, 11)],
 [datetime.date(2022, 10, 11), datetime.date(2022, 10, 11)],
 ['pdf', 'docx', 'pptx'])

## Input of meta-data

In [68]:
# file_size_range_input = 
# creation_time_range_input = 
# modification_time_range_input = 
# filetypes_input = 

# Initial Search using Meta-Data

In [69]:
if len(keyword) != 0:
    init_search = path_df[(path_df['FileName'].str.contains(keyword, case=False)) & 
            (path_df['FileType'].isin(filetypes)) & 
            (path_df['ModifiedTime'].between(modification_time_range[0],modification_time_range[1])) & 
            (path_df['CreationTime'].between(creation_time_range[0],creation_time_range[1])) & 
            (path_df['FileSize'].between(file_size_range[0],file_size_range[1]))
           ]
    print(init_search.shape)
else:
    print(path_df.shape)

(3, 8)


# Deep Search Modeling

In [77]:
deep_search_df = path_df[~path_df.index.isin(init_search.index)]

In [78]:
deep_search_df["InSearchPath"] = r"Dataset/Textset/" + deep_search_df.Textfile_Name

In [79]:
# In search keywords
keyword_processor.add_keyword(keyword)

False

In [80]:
keyword_processor.get_all_keywords()

{'invest': 'invest'}

In [81]:
inmatch_list = []
for x in deep_search_df.InSearchPath:
    if x.split(".")[-2] != "ppt":
        with open(x) as f:
            lines = f.readlines()
            all_world = " ".join(lines)
            ins_result = keyword_processor.extract_keywords(all_world)
            
            if len(ins_result) != 0:
                inmatch_list.append(x)
                print(x)

Dataset/Textset/2021 Crytpocurrencies_Presentation.pdf.txt
Dataset/Textset/Core Real Estate WP.pdf.txt
Dataset/Textset/Cycles in Active Management WP.pdf.txt
Dataset/Textset/Dual Portfolio Framework FINAL.pdf.txt
Dataset/Textset/EM ILBs WP.pdf.txt
Dataset/Textset/FrontierMarkets.pptx.txt
Dataset/Textset/High Yield Bonds FINAL.pdf.txt
Dataset/Textset/Introduction to Private Equity presentation.pptx.txt
Dataset/Textset/Long-Term Treasuries WP.pdf.txt
Dataset/Textset/Manager Alpha FINAL.pdf.txt
Dataset/Textset/MEKETA_Global Equity Mandates.pdf.txt
Dataset/Textset/MEKETA_Portable Alpha.pdf.txt
Dataset/Textset/MEKETA_Understanding China, Part II.pdf.txt
Dataset/Textset/Stable Value WP.pdf.txt


In [86]:
deef_search_df_result = deep_search_df[deep_search_df.InSearchPath.isin(inmatch_list)]

# Attaching Clickable link to Files

In [134]:
temp_path = r"Dataset/Textset/Stable Value WP.pdf.txt"
os.path.basename(temp_path)

'Stable Value WP.pdf.txt'

In [135]:
'<a href="{}">{}</a>'.format(temp_path, os.path.basename(temp_path))

'<a href="Dataset/Textset/Stable Value WP.pdf.txt">Stable Value WP.pdf.txt</a>'

In [164]:
# Function to convert file path into clickable form.
def fun(path):
    
    # returns the final component of a url
    f_url = os.path.basename(path)
      
    # convert the url into link
    return '<a href="{}">{}</a>'.format(path, "Click Here")

In [165]:
temp_path_df = path_df.copy()

In [166]:
temp_path_df['FileLink'] = temp_path_df.Path

In [168]:
temp_path_df = temp_path_df.style.format({'FileLink' : fun})

In [169]:
temp_path_df

,Path,FileName,FileType,ModifiedTime,CreationTime,FileSize,Textfile_Name,FileLink
0,Dataset\101101_Jana - Concentrated Managers2.ppt,101101_Jana - Concentrated Managers2.ppt,ppt,2022-10-11 10:38:05.810308,2022-10-11 05:07:00,1273856,101101_Jana - Concentrated Managers2.ppt.txt,Click Here
1,Dataset\2021 Crytpocurrencies_Presentation.pdf,2021 Crytpocurrencies_Presentation.pdf,pdf,2022-10-11 10:38:05.869076,2022-10-11 05:07:00,838394,2021 Crytpocurrencies_Presentation.pdf.txt,Click Here
2,Dataset\@One-pager How do higher rates slow the economy.docx,@One-pager How do higher rates slow the economy.docx,docx,2022-10-11 10:38:05.616253,2022-10-11 05:07:00,98091,@One-pager How do higher rates slow the economy.docx.txt,Click Here
3,Dataset\@One-pager What is a Bull Market.docx,@One-pager What is a Bull Market.docx,docx,2022-10-11 10:38:05.668125,2022-10-11 05:07:00,122763,@One-pager What is a Bull Market.docx.txt,Click Here
4,Dataset\@One-pager Why Invest Globally_NB.docx,@One-pager Why Invest Globally_NB.docx,docx,2022-10-11 10:38:05.728691,2022-10-11 05:07:00,107097,@One-pager Why Invest Globally_NB.docx.txt,Click Here
5,Dataset\@One-pager Why Invest in Hedge Funds.docx,@One-pager Why Invest in Hedge Funds.docx,docx,2022-10-11 10:38:05.780274,2022-10-11 05:07:00,100012,@One-pager Why Invest in Hedge Funds.docx.txt,Click Here
6,Dataset\Core Real Estate WP.pdf,Core Real Estate WP.pdf,pdf,2022-10-11 10:38:05.899065,2022-10-11 05:07:00,357552,Core Real Estate WP.pdf.txt,Click Here
7,Dataset\Cycles in Active Management WP.pdf,Cycles in Active Management WP.pdf,pdf,2022-10-11 10:38:05.917473,2022-10-11 05:07:00,760476,Cycles in Active Management WP.pdf.txt,Click Here
8,Dataset\Dual Portfolio Framework FINAL.pdf,Dual Portfolio Framework FINAL.pdf,pdf,2022-10-11 10:38:05.933752,2022-10-11 05:07:00,509218,Dual Portfolio Framework FINAL.pdf.txt,Click Here
9,Dataset\EM ILBs WP.pdf,EM ILBs WP.pdf,pdf,2022-10-11 10:38:05.960563,2022-10-11 05:07:00,151407,EM ILBs WP.pdf.txt,Click Here
